In [27]:
from pathlib import Path
import pandas as pd

# Edge mappings are only based on isomorphic graphs from the en gold train dataset
PATH = Path('data/edge_classifier/edge_mappings_2022_05_08_21_07_53.csv')
df = pd.read_csv(PATH)

<ipython-input-27-9efae633b53a>:6: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH)


In [28]:
df

from_node_token from_node_Case from_node_Number  from_node_Person  \
0         person.n.01            Nom             Sing               1.0   
1           time.n.08            NaN              NaN               NaN   
2           joke.v.01            NaN              NaN               NaN   
3           joke.v.01            NaN              NaN               NaN   
4            tom.n.01            NaN             Sing               NaN   
...               ...            ...              ...               ...   
27140     person.n.01            Nom              NaN               2.0   
27141       time.n.08            NaN              NaN               NaN   
27142      sleep.v.01            NaN              NaN               NaN   
27143      sleep.v.01            NaN              NaN               NaN   
27144      sleep.v.01            NaN              NaN               NaN   

      from_node_PronType from_node_lemma from_node_upos from_node_xpos  \
0                    Prs               I           PRON            PRP   
1                    NaN             NaN            NaN            NaN   
2                    NaN            joke           VERB            VBG   
3                    NaN            joke           VERB            VBG   
4                    NaN             Tom          PROPN            NNP   
...                  ...             ...            ...            ...   
27140                Prs             you           PRON            PRP   
27141                NaN             NaN            NaN            NaN   
27142                NaN           sleep           VERB            VBG   
27143                NaN           sleep           VERB            VBG   
27144                NaN           sleep           VERB            VBG   

      to_node_token     label  ... from_node_NumType to_node_Reflex  \
0           speaker       EQU  ...               NaN            NaN   
1               now       TPR  ...               NaN            NaN   
2       person.n.01     Agent  ...               NaN            NaN   
3         time.n.08      Time  ...               NaN            NaN   
4               Tom      Name  ...               NaN            NaN   
...             ...       ...  ...               ...            ...   
27140       speaker       EQU  ...               NaN            NaN   
27141           now       EQU  ...               NaN            NaN   
27142   person.n.01      Time  ...               NaN            NaN   
27143     time.n.08     Agent  ...               NaN            NaN   
27144      she.n.01  Co-Agent  ...               NaN            NaN   

      to_node_VerbForm from_node_Degree to_node_Tense from_node_Gender  \
0                  NaN              NaN           NaN              NaN   
1                  NaN              NaN           NaN              NaN   
2                  NaN              NaN           NaN              NaN   
3                  NaN              NaN           NaN              NaN   
4                  NaN              NaN           NaN              NaN   
...                ...              ...           ...              ...   
27140              NaN              NaN           NaN              NaN   
27141              NaN              NaN           NaN              NaN   
27142              NaN              NaN           NaN              NaN   
27143              NaN              NaN           NaN              NaN   
27144              NaN              NaN           NaN              NaN   

       from_node_Poss to_node_Mood to_node_Voice to_node_Foreign  
0                 NaN          NaN           NaN             NaN  
1                 NaN          NaN           NaN             NaN  
2                 NaN          NaN           NaN             NaN  
3                 NaN          NaN           NaN             NaN  
4                 NaN          NaN           NaN             NaN  
...               ...          ...           ...             ...  
27140

In [29]:
len(df)

27145

In [30]:
df.columns

Index(['from_node_token', 'from_node_Case', 'from_node_Number',
       'from_node_Person', 'from_node_PronType', 'from_node_lemma',
       'from_node_upos', 'from_node_xpos', 'to_node_token', 'label', 'deprel',
       'doc_id', 'from_node_Tense', 'from_node_VerbForm', 'to_node_Case',
       'to_node_Number', 'to_node_Person', 'to_node_PronType', 'to_node_lemma',
       'to_node_upos', 'to_node_xpos', 'from_node_Mood', 'to_node_Gender',
       'from_node_Voice', 'to_node_Degree', 'to_node_NumType', 'to_node_Poss',
       'from_node_NumType', 'to_node_Reflex', 'to_node_VerbForm',
       'from_node_Degree', 'to_node_Tense', 'from_node_Gender',
       'from_node_Poss', 'to_node_Mood', 'to_node_Voice', 'to_node_Foreign'],
      dtype='object')

In [31]:
from synse.sbn_spec import SBNSpec

len(SBNSpec.ROLES) + len(SBNSpec.DRS_OPERATORS)

96

In [32]:
len(df['label'].unique())

75

In [33]:
feature_cols = [
    "deprel",

    "from_node_Case",
    "from_node_Degree",
    "from_node_Gender",
    "from_node_Mood",
    "from_node_Number",
    "from_node_NumType",
    "from_node_Person",
    "from_node_Poss",
    "from_node_PronType",
    "from_node_Tense",
    "from_node_upos",
    "from_node_VerbForm",
    "from_node_Voice",
    "from_node_xpos",

    "to_node_Case",
    "to_node_Degree",
    "to_node_Foreign",
    "to_node_Gender",
    "to_node_Mood",
    "to_node_Number",
    "to_node_NumType",
    "to_node_Person",
    "to_node_Poss",
    "to_node_PronType",
    "to_node_Reflex",
    "to_node_Tense",
    "to_node_upos",
    "to_node_VerbForm",
    "to_node_Voice",
    "to_node_xpos",
]


In [34]:
# TODO: check duplicates here to find possible ambiguous UD -> DRS parsing (at least on node level)
df = df.drop_duplicates(feature_cols)
len(df)

889

In [35]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [36]:
result = encoder.fit_transform(df[feature_cols].values)

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
    ('encoder', encoder),
    ('clf', DecisionTreeClassifier()),
])

In [42]:
pipeline.fit(df[feature_cols].values, df['label'])

Pipeline(steps=[('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('clf', DecisionTreeClassifier())])

In [43]:
pipeline.predict(df[feature_cols].iloc[0].to_numpy().reshape(1, -1))

array(['EQU'], dtype=object)

In [46]:
pipeline.score(df[feature_cols].values, df['label'])

1.0

In [47]:
'a'.split(':')

['a']

In [44]:
import joblib

joblib.dump(pipeline, 'edge_clf.joblib')

['edge_clf.joblib']

In [11]:
len(result[0])

186

In [12]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(result, df['label'])

DecisionTreeClassifier()

In [13]:
clf.score(result, df['label'])

1.0

In [14]:
clf.predict(result[0].reshape(1, -1))

array(['EQU'], dtype=object)

In [21]:
df.iloc[0]

from_node_token        person.n.01
from_node_Case                 Nom
from_node_Number              Sing
from_node_Person               1.0
from_node_PronType             Prs
from_node_lemma                  I
from_node_upos                PRON
from_node_xpos                 PRP
to_node_token              speaker
label                          EQU
deprel                         NaN
doc_id                en/p12/d1711
from_node_Tense                NaN
from_node_VerbForm             NaN
to_node_Case                   NaN
to_node_Number                 NaN
to_node_Person                 NaN
to_node_PronType               NaN
to_node_lemma                  NaN
to_node_upos                   NaN
to_node_xpos                   NaN
from_node_Mood                 NaN
to_node_Gender                 NaN
from_node_Voice                NaN
to_node_Degree                 NaN
to_node_NumType                NaN
to_node_Poss                   NaN
from_node_NumType              NaN
to_node_Reflex      